In [ ]:
%pip install kafka-python
%pip install pandas
%pip install numpy
%pip install requests
%pip install pyspark
import json
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from kafka import KafkaConsumer

In [34]:
consumer = KafkaConsumer("beehiiv-topic", bootstrap_servers='54.173.140.150:9092',
            auto_offset_reset='earliest', enable_auto_commit=True, 
             value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [ ]:
import json
from kafka import KafkaConsumer
import snowflake.connector
import time

# Configure Kafka consumer
consumer = KafkaConsumer(
    'beehiiv-topic',
    bootstrap_servers=['54.173.140.150:9092'],
    group_id='newsletter-simple-group',
    auto_offset_reset='earliest',
    value_deserializer=lambda x: x.decode('utf-8')
)

# Configure Snowflake connection
conn = snowflake.connector.connect(
    user='THEGLASSOFDATA',
    password='Q1a1z1w2s2x2@@',
    account='uedhkgi-zc09730',
    warehouse='NEWSLETTER_WH',
    database='NEWSLETTER_DB',
    schema='NEWSLETTER_SCHEMA'
)

# Create cursor and explicitly set context
cursor = conn.cursor()
cursor.execute("USE WAREHOUSE NEWSLETTER_WH")
cursor.execute("USE DATABASE NEWSLETTER_DB")
cursor.execute("USE SCHEMA NEWSLETTER_SCHEMA")

# Test connection and verify tables exist
try:
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    print(f"Available tables: {tables}")
except Exception as e:
    print(f"Error checking tables: {e}")

print("Starting Kafka consumer for subscriber data...")

# Process messages from Kafka
for count, message in enumerate(consumer):
    try:
        print(f"\nMessage {count}:")
        print(f"Raw value (first 100 chars): {message.value[:100]}...")
        
        # Try to parse JSON
        try:
            subscriber_dict = json.loads(message.value)
            print(f"Parsed subscriber: {subscriber_dict.get('email')} (ID: {subscriber_dict.get('subscriber_id')})")
        except json.JSONDecodeError as e:
            print(f"JSON parsing failed: {e}")
            continue
        
        # Insert or update in Snowflake using MERGE statement
        try:
            cursor.execute("""
            MERGE INTO NEWSLETTER_DB.NEWSLETTER_SCHEMA.SUBSCRIBERS tgt
            USING (SELECT %s as SUBSCRIBER_ID, %s as EMAIL, %s as NAME, 
                        %s as SUBSCRIPTION_STATUS, %s as SIGNUP_DATE,
                        %s as REFERRAL_SOURCE, %s as TAGS) src
            ON tgt.SUBSCRIBER_ID = src.SUBSCRIBER_ID
            WHEN MATCHED THEN
                UPDATE SET 
                    EMAIL = src.EMAIL,
                    NAME = src.NAME,
                    SUBSCRIPTION_STATUS = src.SUBSCRIPTION_STATUS,
                    SIGNUP_DATE = src.SIGNUP_DATE,
                    REFERRAL_SOURCE = src.REFERRAL_SOURCE,
                    TAGS = src.TAGS,
                    INGESTION_TIMESTAMP = CURRENT_TIMESTAMP()
            WHEN NOT MATCHED THEN
                INSERT (SUBSCRIBER_ID, EMAIL, NAME, SUBSCRIPTION_STATUS, 
                        SIGNUP_DATE, REFERRAL_SOURCE, TAGS)
                VALUES (src.SUBSCRIBER_ID, src.EMAIL, src.NAME, src.SUBSCRIPTION_STATUS,
                        src.SIGNUP_DATE, src.REFERRAL_SOURCE, src.TAGS)
            """, (
                subscriber_dict.get('subscriber_id'),
                subscriber_dict.get('email'),
                subscriber_dict.get('name'),
                subscriber_dict.get('subscription_status'),
                subscriber_dict.get('signup_date'),
                subscriber_dict.get('referral_source'),
                subscriber_dict.get('tags')
            ))
            
            # Commit after each insert
            conn.commit()
            print(f"Successfully inserted/updated subscriber in Snowflake")
            
        except Exception as db_error:
            print(f"Database error: {db_error}")
            # Log error to Snowflake
            try:
                cursor.execute("""
                INSERT INTO NEWSLETTER_DB.NEWSLETTER_SCHEMA.ERROR_LOG (ERROR_MESSAGE)
                VALUES (%s)
                """, (str(db_error),))
                conn.commit()
            except Exception as log_error:
                print(f"Error logging to Snowflake: {log_error}")
            
    except Exception as e:
        print(f"Error processing message: {e}")
        continue

print("Finished processing messages")

# Close connections
cursor.close()
conn.close()
consumer.close()